In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

#df = pd.read_csv("./(V1.6)all_products_data_set.csv", usecols=['index','colors'])
#df.insert(2, "color_info", "")
#df.insert(2, "pattern_info","")
#df.head()

In [2]:
import re
def tc(colors: str)->str:
    temp = (colors.lower().removeprefix("{").removesuffix("}"))
    res = re.sub('"', "", re.sub("/",",",temp))
    return res

In [3]:
import json
#f = open("./json_files/color_name_map.json")
#colors = json.load(f)
#len(colors.keys())

In [4]:
df = pd.read_csv("./look.csv")
for row in range(df.shape[0]):
    if not isinstance(df.loc[row]["colors"], str):
        df.loc[row,'colors'] = ""
        continue
    e = df.loc[row]["colors"]
    c = tc(e)
    df.loc[row,'colors'] = c

In [5]:

df.sample(10)
#df.shape[0]

,index,colors,pattern_info,color_info
23024,23024,white,NaN,White (#ffffff)
19311,19311,peach,NaN,Peach (#fdbb8d)
25223,25223,floral,NaN,Floral (#cd839d)
13433,13433,,NaN,NaN
19646,19646,black,NaN,Black (#000000)
15591,15591,honey,NaN,Honey (#f5be05)
19134,19134,black,NaN,Black (#000000)
3978,3978,black,NaN,Black (#000000)
17624,17624,,NaN,NaN
8315,8315,bamboo,NaN,Bamboo (#75ae61)


In [6]:
for row in range(0,1):
    res = set()
    pattern = set()
    colors_str = df.loc[row]["colors"]
    if colors_str == "":
        continue
    colors_list = df.loc[row]["colors"].split(",")
    #print(row, colors_list)
    for item in colors_list:
        URL = "https://colornames.org/search/results/?type=exact&query="
        URL += item
        try:
            page = requests.get(URL)
            soup = BeautifulSoup(page.content, "html.parser")
            results = soup.find_all("a", class_="button is-fullwidth freshButton")
            
            #the first result which has the highest vote
            content = results[0].find("span").text.strip()
            res.add(content)
        except:
            pattern.add(item)
    df.loc[row, "color_info"] = ",".join(res)
    df.loc[row, "pattern_info"] = ",".join(pattern)


In [15]:
df.sample()

,index,colors,pattern_info,color_info
8782,8782,bamboo,NaN,Bamboo (#75ae61)


In [16]:
df.squeeze()
df.to_csv("look.csv", index=False)

